In [ ]:
!pip install flask pyngrok

import sqlite3
from flask import Flask, request, render_template_string
from pyngrok import ngrok
import threading
import random

# Configuración de la app Flask
app = Flask(__name__)

# Hacer que enumerate esté disponible en el entorno de plantillas
app.jinja_env.globals.update(enumerate=enumerate)

# Conexión a la base de datos SQLite
def init_db():
    conn = sqlite3.connect('vocational_test_results.db', check_same_thread=False)
    cursor = conn.cursor()
    # Crear tabla si no existe
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Results (
            dni TEXT PRIMARY KEY,
            nombre TEXT,
            apellido TEXT,
            probabilidad REAL,
            carrera_recomendada TEXT
        )
    ''')
    conn.commit()
    return conn, cursor

conn, cursor = init_db()

# Mapear las letras a descripciones de carreras
career_descriptions = {
    "C": "Administrativas y Contables",
    "H": "Humanísticas y Sociales",
    "A": "Artísticas",
    "S": "Medicina y Ciencias de la Salud",
    "I": "Ingeniería y Computación",
    "D": "Defensa y Seguridad",
    "E": "Ciencias Exactas y Agrarias"
}

# Plantilla HTML para la interfaz de preguntas
html_template = '''
<html lang="es">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0"> <!-- Meta tag para hacer el sitio responsivo -->
  <title>Test Vocacional</title>
  <style>
    /* Fondo de la página con un gradiente que cambia */
    body {
      background: linear-gradient(45deg, #007bff, #00b0ff, #007bff, #00b0ff);
      background-size: 400% 400%;
      animation: gradientAnimation 10s ease infinite;
      margin: 0;
      height: 100vh;
    }

    /* Centra el formulario y establece el ancho */
    form {
      max-width: 600px;
      margin: auto;
      padding: 20px;
      border: 2px solid #333;
      border-radius: 10px;
      background-color: #f9f9f9;
    }

    /* Centra las etiquetas y los inputs */
    .form-group {
      text-align: center;
      margin-bottom: 15px;
    }

    /* Estilo para las preguntas y opciones */
    .question-container {
      text-align: left;
      margin-bottom: 15px;
    }

    /* Estilo para el botón de envío */
    button {
      display: block;
      margin: 20px auto;
      padding: 10px 20px;
      background-color: #4CAF50;
      color: white;
      border: none;
      border-radius: 5px;
      cursor: pointer;
    }

    button:hover {
      background-color: #45a049;
    }

    /* Animación para el cambio de gradiente */
    @keyframes gradientAnimation {
      0% {
        background-position: 0% 50%;
      }
      50% {
        background-position: 100% 50%;
      }
      100% {
        background-position: 0% 50%;
      }
    }

    /* Media query para dispositivos pequeños (móviles) */
    @media screen and (max-width: 600px) {
      form {
        padding: 10px;
        max-width: 100%; /* El formulario ocupa el 100% del ancho en pantallas pequeñas */
      }

      .form-group input {
        width: 100%; /* Los campos de texto ocupan el 100% del ancho */
        margin-bottom: 10px; /* Agregar espacio entre campos */
      }

      .question-container {
        margin-bottom: 10px;
      }

      button {
        width: 100%; /* El botón ocupa el 100% del ancho */
        padding: 15px;
      }
    }
  </style>
</head>
<body>
  <h1 style="text-align: center;">TEST VOCACIONAL</h1>
  <form method="POST">
    <h2 style="text-align: center;">Datos del Estudiante: </h2>
    <div class="form-group">
      <label>DNI: <input type="text" name="dni" required></label>
    </div>
    <div class="form-group">
      <label>Nombre: <input type="text" name="nombre" required></label>
    </div>
    <div class="form-group">
      <label>Apellido: <input type="text" name="apellido" required></label>
    </div>

    {% for i, q in enumerate(questions) %}
      <div class="question-container">
        <p><strong>{{ i + 1 }}. {{ q.question }}</strong></p>
        {% for code, option in q.options.items() %}
          <label>
            <input type="radio" name="answer{{ i }}" value="{{ code }}" required> {{ option }}
          </label><br>
        {% endfor %}
      </div>
    {% endfor %}

    <button type="submit">Enviar</button>
  </form>
</body>
</html>
'''

# Plantilla HTML para la página de resultado
result_template = '''
<!DOCTYPE html>
<html lang="es">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0"> <!-- Meta tag para hacer el sitio responsivo -->
  <title>Resultado del Test Vocacional</title>
  <style>
    /* Fondo de la página con un gradiente que cambia */
    body {
      background: linear-gradient(45deg, #007bff, #00b0ff, #007bff, #00b0ff);
      background-size: 400% 400%;
      animation: gradientAnimation 10s ease infinite;
      margin: 0;
      height: 100vh;
      display: flex;
      justify-content: center;
      align-items: center;
    }

    /* Contenedor principal del resultado */
    .result-container {
      max-width: 600px;
      width: 100%;
      padding: 20px;
      border: 2px solid #333;
      border-radius: 10px;
      background-color: #f9f9f9;
      text-align: center;
    }

    h1, h2 {
      color: #333;
    }

    /* Estilo para el botón de repetir el test */
    button {
      display: block;
      margin: 20px auto;
      padding: 10px 20px;
      background-color: #4CAF50;
      color: white;
      border: none;
      border-radius: 5px;
      cursor: pointer;
    }

    button:hover {
      background-color: #45a049;
    }

    /* Animación para el cambio de gradiente */
    @keyframes gradientAnimation {
      0% {
        background-position: 0% 50%;
      }
      50% {
        background-position: 100% 50%;
      }
      100% {
        background-position: 0% 50%;
      }
    }

    /* Media query para dispositivos pequeños (móviles) */
    @media screen and (max-width: 600px) {
      .result-container {
        padding: 15px;
        width: 90%; /* El contenedor ocupa el 90% del ancho en pantallas pequeñas */
      }

      button {
        width: 100%; /* El botón ocupa el 100% del ancho */
        padding: 15px;
      }
    }
  </style>
</head>
<body>
  <div class="result-container">
    <h1>Resultado del Test Vocacional</h1>
    <h2>Carrera recomendada: {{ career_descriptions[result] }}</h2>
    <form action="/" method="get">
      <button type="submit">Repetir el Test</button>
    </form>
  </div>
</body>
</html>
'''

# Definir preguntas y opciones
questions = [
    {
        "question": "¿Disfrutas trabajando con cifras, balances y temas financieros?",
        "options": {"C": "Sí, es algo que me interesa mucho", "H": "Prefiero entender los aspectos sociales de las personas", "A": "No, prefiero el lado creativo"}
    },
    {
        "question": "¿Te interesa comprender el funcionamiento de la naturaleza y los procesos científicos?",
        "options": {"E": "Sí, me gustaría investigar en ciencias exactas o agrícolas", "S": "Me interesa más la investigación en salud", "I": "Prefiero estudiar y desarrollar tecnología"}
    },
    {
        "question": "¿Te imaginas dirigiendo un equipo en la realización de proyectos?",
        "options": {"C": "Sí, me gustaría planificar y liderar proyectos", "I": "Sí, pero me gusta más el enfoque técnico", "H": "Prefiero coordinar con aspectos humanos y sociales"}
    },
    {
        "question": "¿Te sientes atraído por las artes como la música, el diseño o el teatro?",
        "options": {"A": "Sí, me apasionan las artes", "H": "Me gusta más entender cómo influyen en la sociedad", "C": "Prefiero enfocarme en su gestión o producción"}
    },
    {
        "question": "¿Considerarías una carrera en defensa nacional o seguridad?",
        "options": {"D": "Sí, me gustaría contribuir a la seguridad", "I": "Podría interesarme en tecnología de seguridad", "H": "Me interesa más la mediación o diplomacia"}
    },
    {
        "question": "¿Te interesa el cuidado y mejora de la salud humana?",
        "options": {"S": "Sí, me gustaría trabajar en salud", "E": "Prefiero investigar biomecánica o ciencia", "H": "Me interesa más en su impacto social"}
    },
    {
        "question": "¿Te gustaría resolver problemas complejos en el mundo de la tecnología?",
        "options": {"I": "Sí, me encanta la idea de desarrollar soluciones tecnológicas", "C": "Prefiero resolver problemas de gestión", "E": "Me interesa más la parte teórica y de investigación"}
    },
    {
        "question": "¿Te gusta trabajar con tus manos y construir cosas?",
        "options": {"I": "Sí, disfruto diseñando y construyendo", "D": "Prefiero construir en contextos de seguridad y defensa", "A": "Me gusta crear cosas artísticas"}
    },
    {
        "question": "¿Te resulta interesante analizar el comportamiento humano y las interacciones sociales?",
        "options": {"H": "Sí, me gustaría trabajar en contextos sociales", "C": "Prefiero analizar aspectos económicos", "A": "Me interesa cómo el arte refleja la sociedad"}
    },
    {
        "question": "¿Disfrutas investigando sobre el universo y las leyes de la física?",
        "options": {"E": "Sí, quisiera explorar más sobre el universo", "I": "Estoy más interesado en aplicar esos conceptos", "S": "Me atrae más cómo afecta a la biología"}
    },
    {
        "question": "¿Te gustaría trabajar protegiendo a otros y garantizando su seguridad?",
        "options": {"D": "Sí, quiero trabajar en seguridad", "S": "Prefiero la seguridad en el contexto de la salud", "H": "Me gustaría más ser mediador en conflictos"}
    },
    {
        "question": "¿Escribirías o relatarías historias para transmitir ideas y emociones?",
        "options": {"A": "Sí, me interesa la narrativa", "H": "Prefiero la comunicación a través de medios sociales", "C": "Me interesa relatar casos de éxito empresarial"}
    },
    {
        "question": "¿Te gustaría trabajar en un laboratorio investigando nuevos descubrimientos?",
        "options": {"E": "Sí, quiero descubrir cosas nuevas", "S": "Me interesa más el lado de la bioquímica", "I": "Prefiero desarrollar prototipos tecnológicos"}
    },
    {
        "question": "¿Te interesa la gestión de equipos y el liderazgo empresarial?",
        "options": {"C": "Sí, quiero liderar a otros", "I": "Me gusta más liderar proyectos técnicos", "H": "Prefiero liderar en entornos sociales"}
    },
    {
        "question": "¿Te interesa innovar en el campo de la ingeniería?",
        "options": {"I": "Sí, quiero estar en la vanguardia de la tecnología", "E": "Quiero investigar en teorías físicas", "D": "Prefiero desarrollar soluciones para la defensa"}
    },
    {
        "question": "¿Te atrae el mundo del cine y la producción audiovisual?",
        "options": {"A": "Sí, me gustaría participar en ese mundo", "H": "Prefiero cómo se analiza socialmente el cine", "C": "Me interesa más su rentabilidad y producción"}
    },
    {
        "question": "¿Te encuentras ayudando a otros a resolver sus problemas emocionales o sociales?",
        "options": {"H": "Sí, quiero ayudar a otros socialmente", "S": "Prefiero el contexto de la salud mental", "D": "Me interesa más en contextos de crisis de defensa"}
    },
    {
        "question": "¿Te gustaría trabajar en la protección de recursos naturales y el medio ambiente?",
        "options": {"E": "Sí, quiero trabajar en conservación", "S": "Me interesa más la salud pública relacionada", "I": "Prefiero innovar en tecnología ambiental"}
    },
    {
        "question": "¿Aspiras a diseñar soluciones arquitectónicas innovadoras?",
        "options": {"I": "Sí, me encanta el diseño arquitectónico", "A": "Prefiero el aspecto estético y artístico", "C": "Me gustaría gestionar proyectos arquitectónicos"}
    },
    {
        "question": "¿Te interesa estudiar la historia y su impacto en el presente?",
        "options": {"H": "Sí, quiero analizar contextos históricos", "A": "Me gustaría explorar su influencia en la cultura", "D": "Me interesa más en estrategia militar histórica"}
    },
    {
        "question": "¿Te gustaría asesorar en la creación de políticas de salud pública?",
        "options": {"S": "Sí, me interesa mejorar la salud pública", "E": "Prefiero las bases científicas de estas políticas", "H": "Me interesa más cómo afectan a la sociedad"}
    },
    {
        "question": "¿Te sientes a gusto solucionando problemas matemáticos complejos?",
        "options": {"E": "Sí, los disfruto mucho", "C": "Prefiero aplicarlos en contextos financieros", "I": "Me interesan los problemas computacionales"}
    },
    {
        "question": "¿Te interesa entender los comportamientos de los organismos vivos?",
        "options": {"S": "Sí, estoy fascinado por la biología", "E": "Me interesa más en el contexto de la evolución", "H": "Prefiero entender sus interacciones sociales"}
    },
    {
        "question": "¿Te gustaría realizar investigaciones científicas básicas?",
        "options": {"E": "Sí, estoy interesado en la ciencia básica", "S": "Me interesa más la aplicación médica", "I": "Prefiero el enfoque de tecnologías aplicadas"}
    },
    {
        "question": "¿Te gustaría trabajar en organización de eventos y producción de shows?",
        "options": {"A": "Sí, me apasiona la producción creativa", "C": "Prefiero la logística y administración", "H": "Me interesa más su impacto cultural"}
    },
    {
        "question": "¿Te interesa explorar el espacio y los planetas?",
        "options": {"E": "Sí, el espacio me fascina", "I": "Prefiero diseñar tecnología espacial", "A": "Estoy más interesado en la astrofotografía"}
    },
    {
        "question": "¿Te gusta analizar datos estadísticos para encontrar patrones?",
        "options": {"C": "Sí, quiero usar datos en finanzas", "E": "Me gusta aplicarlos en investigaciones científicas", "I": "Prefiero su uso en inteligencia artificial"}
    },
    {
        "question": "¿Quieres trabajar en el diseño de videojuegos?",
        "options": {"I": "Sí, quiero programar y crear juegos", "A": "Me interesa el diseño visual y artístico", "C": "Prefiero gestionar proyectos de juegos"}
    },
    {
        "question": "¿Te involucras en la defensa de los derechos humanos?",
        "options": {"H": "Sí, quiero trabajar en impacto social", "D": "Me interesa en la defensa legal", "A": "Prefiero campañas y comunicaciones creativas"}
    },
    {
    "question": "¿Te interesa trabajar en el desarrollo de energías renovables?",
    "options": {"E": "Sí, quiero contribuir al desarrollo sostenible", "I": "Me gustaría innovar en tecnologías energéticas", "S": "Me interesa más la aplicación de estas tecnologías en salud"}
    }

]

@app.route("/", methods=["GET", "POST"])
def vocational_test():
    if request.method == "POST":
        # Obtener datos del usuario
        dni = request.form["dni"]
        nombre = request.form["nombre"]
        apellido = request.form["apellido"]

        # Obtener respuestas y calcular probabilidad
        scores = {"C": 0, "H": 0, "A": 0, "S": 0, "I": 0, "D": 0, "E": 0}
        total_questions = len(questions)

        for i in range(total_questions):
            answer = request.form.get(f"answer{i}")
            if answer:
                scores[answer] += 1

        # Calcular la carrera con la máxima puntuación
        max_score = max(scores.values())
        possible_careers = [career for career, score in scores.items() if score == max_score]
        recommended_career = random.choice(possible_careers)
        probability = (max_score / total_questions) * 100

        try:
            # Guardar datos en la base de datos
            cursor.execute('''
                INSERT OR REPLACE INTO Results (dni, nombre, apellido, probabilidad, carrera_recomendada)
                VALUES (?, ?, ?, ?, ?)
            ''', (dni, nombre, apellido, probability, career_descriptions[recommended_career]))
            conn.commit()
        except Exception as e:
            print(f"Error al guardar en la base de datos: {e}")

        # Redirigir al resultado
        return render_template_string(result_template, career_descriptions=career_descriptions, result=recommended_career)

    return render_template_string(html_template, questions=questions, career_descriptions=career_descriptions)
# Función para visualizar los resultados almacenados en la base de datos
def view_results():
    # Obtener todos los registros de la tabla Results
    cursor.execute("SELECT * FROM Results")
    rows = cursor.fetchall()

    # Imprimir cada registro
    for row in rows:
        print(f"DNI: {row[0]}, Nombre: {row[1]}, Apellido: {row[2]}, Probabilidad: {row[3]}, Carrera Recomendada: {row[4]}")

# Función para iniciar ngrok y obtener la URL pública
def start_ngrok():
    # Reemplaza "TU_AUTHTOKEN_NGROK" con tu authtoken de ngrok
    authtoken = "2jV6pwxYItpUrJqIci4umM437xT_2TX7rh5zxuwYakVDoak2v"
    ngrok.set_auth_token(authtoken)

    public_url = ngrok.connect(5000)
    print(f" * ngrok URL: {public_url}")

# Ejecutar ngrok en un hilo separado para evitar bloquear Flask
threading.Thread(target=start_ngrok).start()

# Iniciar la app Flask
app.run(port=5000)

# Llama a la función para ver los resultados
view_results()